# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 100 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
#импорт всех необходимых библиотек
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [2]:
# количество точек для разведки
POINTS_TO_EXPLORE = 500

# количество точек для разработки
POINTS_TO_BUILD = 200

# бюджет на разработку скважин в регионе
BUDGET = 10000000000

# доход с каждой единицы продукта
REVENUE_PER_PRODUCT = 450000

# пороговая вероятность убытков
LOSS_RATE = 0.025

## Загрузка и подготовка данных

In [3]:
#загрузим даннные в переменные 
data_1 = pd.read_csv('/datasets/geo_data_0.csv')
data_2 = pd.read_csv('/datasets/geo_data_1.csv')
data_3 = pd.read_csv('/datasets/geo_data_2.csv')

In [4]:
#познакомимся с данными
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [5]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305


In [6]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


In [7]:
data_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [8]:
data_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [9]:
data_3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [10]:
#проверка на пропуски
data = [data_1, data_2, data_3]
for i in data:
    print(i.isna().sum())
    print('\n')

id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64


id         0
f0         0
f1         0
f2         0
product    0
dtype: int64




In [11]:
#проверка на явные дубликаты
data = [data_1, data_2, data_3]
for i in data:
    print(i.duplicated().sum())
    print('\n')

0


0


0




In [12]:
#округлим все значения до двух знаков после запятой
data_1[['f0', 'f1', 'f2', 'product']] = data_1[['f0', 'f1', 'f2', 'product']].round(2)
data_2[['f0', 'f1', 'f2', 'product']] = data_2[['f0', 'f1', 'f2', 'product']].round(2)
data_3[['f0', 'f1', 'f2', 'product']] = data_3[['f0', 'f1', 'f2', 'product']].round(2)

In [13]:
#проверим преобразование
data_1.head()

,id,f0,f1,f2,product
0,txEyH,0.71,-0.50,1.22,105.28
1,2acmU,1.33,-0.34,4.37,73.04
2,409Wp,1.02,0.15,1.42,85.27
3,iJLyR,-0.03,0.14,2.98,168.62
4,Xdl7t,1.99,0.16,4.75,154.04


In [14]:
data_2.head()

,id,f0,f1,f2,product
0,kBEdx,-15.00,-8.28,-0.01,3.18
1,62mP7,14.27,-3.48,1.00,26.95
2,vyE1P,6.26,-5.95,5.00,134.77
3,KcrkZ,-13.08,-11.51,5.00,137.95
4,AHL4O,12.70,-8.15,5.00,134.77


In [15]:
data_3.head()

,id,f0,f1,f2,product
0,fwXo0,-1.15,0.96,-0.83,27.76
1,WJtFt,0.26,0.27,-2.53,56.07
2,ovLUW,0.19,0.29,-5.59,62.87
3,q6cA6,2.24,-0.55,0.93,114.57
4,WPMUX,-0.52,1.72,5.90,149.60


## Обучение и проверка модели

In [16]:
RANDOM_STATE = 42

X_train_1, X_val_1 = train_test_split(data_1, test_size=0.25, random_state=42)
X_train_2, X_val_2 = train_test_split(data_2, test_size=0.25, random_state=42)
X_train_3, X_val_3 = train_test_split(data_3, test_size=0.25, random_state=42)

y_train_1 = X_train_1['product']
y_train_2 = X_train_2['product']
y_train_3 = X_train_3['product']

y_true_1 = X_val_1['product']
y_true_2 = X_val_2['product']
y_true_3 = X_val_3['product']

In [17]:
X_train_1 = X_train_1.drop(['id', 'product'], axis=1)
X_train_2 = X_train_2.drop(['id', 'product'], axis=1)
X_train_3 = X_train_3.drop(['id', 'product'], axis=1)

In [18]:
# Создание моделей
model_1 = LinearRegression()
model_2 = LinearRegression()
model_3 = LinearRegression()

In [19]:
# Обучение моделей
model_1.fit(X_train_1, y_train_1)
model_2.fit(X_train_2, y_train_2)
model_3.fit(X_train_3, y_train_3)

LinearRegression()

In [20]:
X_val_1 = X_val_1.drop(['id', 'product'], axis=1)
X_val_2 = X_val_2.drop(['id', 'product'], axis=1)
X_val_3 = X_val_3.drop(['id', 'product'], axis=1)

In [21]:
y_pred_1 = model_1.predict(X_val_1)
y_pred_2 = model_2.predict(X_val_2)
y_pred_3 = model_3.predict(X_val_3)

In [22]:
RMSE_1 = mean_squared_error(y_true_1, y_pred_1, squared=False)
RMSE_2 = mean_squared_error(y_true_2, y_pred_2, squared=False)
RMSE_3 = mean_squared_error(y_true_3, y_pred_3, squared=False)

In [23]:
dataframe = {
    'predicted_product_1_mean' : y_pred_1.mean(),
    'predicted_product_2_mean' : y_pred_2.mean(),
    'predicted_product_3_mean' : y_pred_3.mean(),
    'RMSE_1' : RMSE_1,
    'RMSE_2' : RMSE_2,
    'RMSE_3' : RMSE_3
}

In [24]:
predictions = pd.DataFrame(dataframe, index=[0])

In [25]:
#вывод на экране средний запас предсказанного сырья и RMSE моделей.
predictions

,predicted_product_1_mean,predicted_product_2_mean,predicted_product_3_mean,RMSE_1,RMSE_2,RMSE_3
0,92.398777,68.711856,94.771112,37.756641,0.893389,40.145569


**Вывод:**

Согласно модели средние предсказанные значения равны 92.4, 68.7, 94.8 тысяч баррелей для первого, второго и третьего региона соотвественно. Значение метрики RMSE указывает на то, что модели, обученные на первом и третьем регионе значительно уступают в точности предсказания, в отличии от модели, обученной на втором регионе.

## Подготовка к расчёту прибыли

**Рассчитайте достаточный объём сырья для безубыточной разработки новой скважины.**

При разведке региона исследуют 500 точек, из которых с помощью машинного обучения выбирают 200 лучших для разработки.

При нынешних ценах один баррель сырья приносит 450 рублей дохода. Доход с каждой единицы продукта составляет 450 тыс. рублей, поскольку объём указан в тысячах баррелей.

Бюджет на разработку скважин в регионе — 10 млрд рублей.

In [26]:
#рассчитаем достаточный объём сырья для безубыточной разработки новой скважины.
enough_volume = BUDGET/POINTS_TO_BUILD/REVENUE_PER_PRODUCT
print(f'Достаточный объем сырья для безубыточной разработки новой скважины равен {round(enough_volume, 2)} тысяч баррелей')

Достаточный объем сырья для безубыточной разработки новой скважины равен 111.11 тысяч баррелей


**Сравните полученный объём сырья со средним запасом в каждом регионе.**

In [27]:
print(f'Cредний запас скважин в регионе 1 равен {round(data_1["product"].mean(), 2)} тыс. баррелей, достаточный объем - {round(enough_volume, 2)}')
print(f'Cредний запас скважин в регионе 2 равен {round(data_2["product"].mean(), 2)} тыс. баррелей, достаточный объем - {round(enough_volume, 2)}')
print(f'Cредний запас скважин в регионе 3 равен {round(data_3["product"].mean(), 2)} тыс. баррелей, достаточный объем - {round(enough_volume, 2)}')

Cредний запас скважин в регионе 1 равен 92.5 тыс. баррелей, достаточный объем - 111.11
Cредний запас скважин в регионе 2 равен 68.83 тыс. баррелей, достаточный объем - 111.11
Cредний запас скважин в регионе 3 равен 95.0 тыс. баррелей, достаточный объем - 111.11


<div class="alert alert-block alert-success">
<b>Успех:</b> Точка безубыточности найдена верно, сравнение проведено!
</div>

In [28]:
#напишем функцию для рассчета прибыли
def revenue(sample_test, sample_prediction):
    # отбираем топ-N точек по запасам сырья
    prediction = sample_prediction.sort_values(by='product', ascending=False)
    product_sum = sample_test.loc[prediction.index][:200].sum()

    # считаем прибыль
    return product_sum * REVENUE_PER_PRODUCT - BUDGET

**Вывод по разделу:**

Средний запас сырья значительно ниже достаточного объема по всем регионам.

## Расчёт прибыли и рисков 

**Напишите функцию для расчёта прибыли по выбранным скважинам и предсказаниям модели:**

In [29]:
#превратим предсказанные значения в датафреймы, а также свяжем индексы
y_pred_1_df = pd.DataFrame(y_pred_1, columns=['product'])
y_pred_2_df = pd.DataFrame(y_pred_2, columns=['product'])
y_pred_3_df = pd.DataFrame(y_pred_3, columns=['product'])

In [30]:
#присвоим индексы новым датафреймам
y_pred_1_df.index = y_true_1.index
y_pred_2_df.index = y_true_2.index
y_pred_3_df.index = y_true_3.index

In [31]:
#посмотрим результаты работы
y_pred_1_df.head()

,product
75721,101.929265
80184,78.160179
19864,115.193432
76699,105.671091
92991,98.049732


In [32]:
y_pred_2_df.head()

,product
75721,0.764303
80184,52.840504
19864,135.107080
76699,109.504547
92991,-0.102358


In [33]:
y_pred_3_df.head()

,product
75721,98.287963
80184,101.587447
19864,52.467355
76699,109.907007
92991,72.426478


In [34]:
#превратим все настоящие значения в датафрейм
y_true_1 = pd.DataFrame(y_true_1)
y_true_2 = pd.DataFrame(y_true_2)
y_true_3 = pd.DataFrame(y_true_3)

In [35]:
sample_test_1 = y_true_1.sample(n=500, replace=True, random_state=RANDOM_STATE)
sample_prediction_1 = y_pred_1_df.loc[sample_test_1.index]
income_1 = revenue(sample_test_1, sample_prediction_1)

In [36]:
sample_test_2 = y_true_2.sample(n=500, replace=True, random_state=RANDOM_STATE)
sample_prediction_2 = y_pred_2_df.loc[sample_test_1.index]
income_2 = revenue(sample_test_2, sample_prediction_2)

In [37]:
sample_test_3 = y_true_3.sample(n=500, replace=True, random_state=RANDOM_STATE)
sample_prediction_3 = y_pred_3_df.loc[sample_test_1.index]
income_3 = revenue(sample_test_3, sample_prediction_3)

In [38]:
print(f'Прибыль для полученного обьема сырья для региона 1 равна {int(income_1)}')
print(f'Прибыль для полученного обьема сырья для региона 2 равна {int(income_2)}')
print(f'Прибыль для полученного обьема сырья для региона 3 равна {int(income_3)}')

Прибыль для полученного обьема сырья для региона 1 равна 571791500
Прибыль для полученного обьема сырья для региона 2 равна 763883000
Прибыль для полученного обьема сырья для региона 3 равна 9498500


In [39]:
#для региона 1
revenues_per_region_1 = []
for i in range(1000):
    target_subsample = y_true_1.sample(n=500, replace=True)
    target_prediction = y_pred_1_df.loc[target_subsample.index]
    revenues_per_region_1.append(revenue(target_subsample, target_prediction))

# Преобразуем список в датафрейм после завершения цикла
revenues_per_region_1 = pd.DataFrame(revenues_per_region_1)
#посчитаем убыток
negative_revenue_probability_1 = (revenues_per_region_1['product'] < 0).mean()

#посчитаем интервал и среднюю
lower_1 = revenues_per_region_1['product'].quantile(0.025)
higher_1 = revenues_per_region_1['product'].quantile(0.975)
mean_1 = revenues_per_region_1['product'].mean()

print(f'Средняя прибыль:', mean_1)
print(f'2.5%-квантиль для:', lower_1)
print(f'97.5%-квантиль для:', higher_1)
print(f'Доля случаев, когда прибыль была отрицательной', negative_revenue_probability_1)

Средняя прибыль: 430683776.00000006
2.5%-квантиль для: -102145487.50000004
97.5%-квантиль для: 961154900.0
Доля случаев, когда прибыль была отрицательной 0.054


In [40]:
#для региона 2
revenues_per_region_2 = []
for i in range(1000):
    target_subsample_2 = y_true_2.sample(n=500, replace=True)
    target_prediction_2 = y_pred_2_df.loc[target_subsample_2.index]
    revenues_per_region_2.append(revenue(target_subsample_2, target_prediction_2))

# Преобразуем список в датафрейм после завершения цикла
revenues_per_region_2 = pd.DataFrame(revenues_per_region_2)
#посчитаем убыток
negative_revenue_probability_2 = (revenues_per_region_2['product'] < 0).mean()

#посчитаем интервал и среднюю
lower_2 = revenues_per_region_2['product'].quantile(0.025)
higher_2 = revenues_per_region_2['product'].quantile(0.975)
mean_2 = revenues_per_region_2['product'].mean()

print(f'Средняя прибыль:', mean_2)
print(f'2.5%-квантиль для:', lower_2)
print(f'97.5%-квантиль для:', higher_2)
print(f'Доля случаев, когда прибыль была отрицательной', negative_revenue_probability_2)

Средняя прибыль: 501401968.9999984
2.5%-квантиль для: 56095437.499999955
97.5%-квантиль для: 964389162.4999981
Доля случаев, когда прибыль была отрицательной 0.013


In [41]:
#для региона 3
revenues_per_region_3 = []
for i in range(1000):
    target_subsample_3 = y_true_3.sample(n=500, replace=True)
    target_prediction_3 = y_pred_3_df.loc[target_subsample_3.index]
    revenues_per_region_3.append(revenue(target_subsample_3, target_prediction_3))

# Преобразуем список в датафрейм после завершения цикла
revenues_per_region_3 = pd.DataFrame(revenues_per_region_3)
#посчитаем убыток
negative_revenue_probability_3 = (revenues_per_region_3['product'] < 0).mean()

#посчитаем интервал и среднюю
lower_3 = revenues_per_region_3['product'].quantile(0.025)
higher_3 = revenues_per_region_3['product'].quantile(0.975)
mean_3 = revenues_per_region_3['product'].mean()

print(f'Средняя прибыль:', mean_3)
print(f'2.5%-квантиль для:', lower_3)
print(f'97.5%-квантиль для:', higher_3)
print(f'Доля случаев, когда прибыль была отрицательной', negative_revenue_probability_3)

Средняя прибыль: 401786544.5
2.5%-квантиль для: -139375562.49999997
97.5%-квантиль для: 919282737.5
Доля случаев, когда прибыль была отрицательной 0.067


**Итоговый вывод:**
Для разработки скважин лучше всего подойдет регион 2, так как средняя прибыль там самая высокая, 95% доверительный интервал высокий, а доля случаев, когда прибыль была отрицательной минимальна. 

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  Jupyter Notebook открыт
- [x]  Весь код выполняется без ошибок
- [x]  Ячейки с кодом расположены в порядке исполнения
- [x]  Выполнен шаг 1: данные подготовлены
- [x]  Выполнен шаг 2: модели обучены и проверены
    - [x]  Данные корректно разбиты на обучающую и валидационную выборки
    - [x]  Модели обучены, предсказания сделаны
    - [x]  Предсказания и правильные ответы на валидационной выборке сохранены
    - [x]  На экране напечатаны результаты
    - [x]  Сделаны выводы
- [x]  Выполнен шаг 3: проведена подготовка к расчёту прибыли
    - [x]  Для всех ключевых значений созданы константы Python
    - [x]  Посчитано минимальное среднее количество продукта в месторождениях региона, достаточное для разработки
    - [x]  По предыдущему пункту сделаны выводы
    - [x]  Написана функция расчёта прибыли
- [x]  Выполнен шаг 4: посчитаны риски и прибыль
    - [x]  Проведена процедура *Bootstrap*
    - [x]  Все параметры бутстрепа соответствуют условию
    - [x]  Найдены все нужные величины
    - [x]  Предложен регион для разработки месторождения
    - [x]  Выбор региона обоснован